In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

from datetime import date

In [2]:
html = urlopen("http://mse.mn/mn/company/458")
soup = BeautifulSoup(html)

In [3]:
soup.title

<title>Монголын Хөрөнгийн Бирж - Таван толгой ХК </title>

In [5]:
soup.title.text.strip()

'Монголын Хөрөнгийн Бирж - Таван толгой ХК'

In [20]:
df = pd.DataFrame()
for row in soup.find_all("div", class_="company_finance_item"):
    col_name = row["id"][8:]
    row_data = row.find_all("li")
    dic = {}
    if row_data:
        for el in row_data:
            s = el.text.split()
            str_vals = []
            for c in s:
                if c.isalpha() or '/' in c or ')' in c:
                    str_vals.append(c)
                else:
                    if any(q.isalpha() for q in c):
                        temp = ""
                        for i in range(len(c)):
                            if c[i].isdigit():
                                str_vals.append(c[:i])
                                num = c[i:]
                                break
                    else:
                        num = c
            dic[" ".join(str_vals)] = float(num.replace(',',''))
        new_df = pd.DataFrame(dic.values(), index=dic.keys(), columns=[col_name])
        df = pd.concat([new_df, df], axis=1, sort=False)

In [22]:
df.to_csv("TTL_finance_ratio.csv")